In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('./modified_seeds_dataset2.txt.csv', sep="\t")
inp_features = ["f1", "f2", "f3", "f4", "f5", "f6", "f7"]
df.head()

,f1,f2,f3,f4,f5,f6,f7,type
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [3]:
scaled_df = df[inp_features].apply(lambda x:x/(x.max() - x.min()))
scaled_df = pd.concat([scaled_df, df[["type"]]], axis = 1)
scaled_df.head()

,f1,f2,f3,f4,f5,f6,f7,type
0,1.440982,3.066116,7.903811,3.244932,2.360656,0.288783,2.570162,1
1,1.405099,3.010331,7.995463,3.127252,2.375624,0.132364,2.440177,1
2,1.349386,2.911157,8.212341,2.979167,2.378475,0.350934,2.375677,1
3,1.306893,2.880165,8.126134,2.997748,2.408411,0.293724,2.365830,1
4,1.524079,3.097107,8.197822,3.185811,2.538845,0.176182,2.548006,1


In [4]:
from sklearn.utils import shuffle
scaled_df = shuffle(scaled_df)
scaled_df.head()

,f1,f2,f3,f4,f5,f6,f7,type
133,1.525968,3.167355,7.843920,3.291104,2.419815,0.554682,2.853274,2
161,1.135033,2.754132,7.715971,3.012387,2.002851,0.555332,2.613491,3
85,1.725212,3.324380,8.049002,3.475788,2.602281,0.317648,3.051206,2
149,1.018886,2.671488,7.356624,2.993806,1.887384,0.710190,2.557361,3
53,1.353163,2.950413,8.013612,3.099099,2.280114,0.432719,2.572132,1


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test_val, y_train, y_test_val = train_test_split(scaled_df[inp_features], 
                                                            scaled_df[["type"]], train_size=0.6, shuffle=False)
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, shuffle=False)
print(len(X_train), len(y_train), len(X_test), len(y_test), len(X_val), len(y_val))

126 126 42 42 42 42


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

In [13]:
class my_model(nn.Module):
    '''A simple model with 1 hidden layer. 7,10 and 3 neurons in input, hidden and output layer respectively.'''
    def __init__(self): 
        super(my_model, self).__init__()
        self.input_layer = nn.Linear(in_features=7, out_features=10)
        self.output_layer = nn.Linear(in_features=10, out_features=3)
    
    def forward(self, x1):
        output = self.output_layer(F.relu(self.input_layer(x1)))
        return output
    
model = my_model()  
summary(model, (1,7))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 10]              80
            Linear-2                 [-1, 1, 3]              33
Total params: 113
Trainable params: 113
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------
